In [156]:
using Statistics

In [2]:
include("ED_functions.jl")

Build_Correlation_Matrix_wAncillas (generic function with 1 method)

In [148]:
function Build_H(N, Δ, μ, E, J = 1.0)
    #J: Hopping amplitude.
    #Δ: Nearest-neighbor density-density interaction.
    #μ: Chemical potential of the diode.
    #E: Tilt strength.

    H = zeros(2^N, 2^N)
    
    for i=1:N-1 #H_S
        H += 0.5*J*Build_C_dagi_Cj_Matrix(N, i, i+1)
        H += 0.5*J*Build_C_dagi_Cj_Matrix(N, i+1, i)
        H += Δ*(Build_C_dagi_Cj_Matrix(N, i, i)-0.5*Enlarge_Matrix_site_j(N, 1, Identity))*(Build_C_dagi_Cj_Matrix(N, i+1, i+1)-0.5*Enlarge_Matrix_site_j(N, 1, Identity))
    end

    for i=1:N
        H += (μ + 0.5*E*i)*(Build_C_dagi_Cj_Matrix(N, i, i)-0.5*Enlarge_Matrix_site_j(N, 1, Identity))
    end
        
    return H
end

function Build_JP_persite_operators(N, J = 1.0)
    #J: Hopping amplitude.
    
    JP_operators = []
    for i=1:N-1
        JP_i = 1im*J*0.5*(Build_C_dagi_Cj_Matrix(N, i, i+1) - Build_C_dagi_Cj_Matrix(N, i+1, i)) #J_i = i(J/2)*(c†i*ci+1 - c†i+1*ci)
        push!(JP_operators, JP_i)
    end

    return JP_operators
end

function Build_Liouvillian_Superoperator(H, gamma_k_list, L_k_list)

    N = log2(size(H)[1])
    superIdentity = Enlarge_Matrix_site_j(N, 1, Identity)
    
    superH = -1im*(kron(superIdentity, H) - kron(transpose(H), superIdentity))
    superL = zeros(size(superH))
    
    for k=1:length(gamma_k_list)
        L_k = L_k_list[k]
        gamma_k = gamma_k_list[k]
        superL = superL + gamma_k*(kron(conj(L_k), L_k) -0.5*(kron(superIdentity, adjoint(L_k)*L_k) + kron(transpose(L_k)*conj(L_k), superIdentity)))
    end

    Liouvillian_Superoperator = superH + superL
    return Liouvillian_Superoperator
end

function Lindblad_Master_Equation(rho, H, gamma_k_list, L_k_list)

    H_Term = -1im*(H*rho - rho*H)
    Dissipation_Term = zeros(size(H_Term))

    for k=1:length(gamma_k_list)
        L_k = L_k_list[k]
        gamma_k = gamma_k_list[k]
        Dissipation_Term = Dissipation_Term + gamma_k*(L_k*rho*adjoint(L_k) - 0.5*(adjoint(L_k)*L_k*rho + rho*adjoint(L_k)*L_k))
    end

    return H_Term + Dissipation_Term
end

Lindblad_Master_Equation (generic function with 1 method)

In [149]:
N = 4
Δ = 5

Γ = 1.0
f_values = 1.0 #f>0 forward, f<0 reverse

JP_operators = Build_JP_persite_operators(N); 

In [150]:
#Same operators for all E.

JP_operators = Build_JP_persite_operators(N); 

L_1_plus = Build_Cdag_Matrix(N, 1)
L_1_minus = Build_C_Matrix(N, 1)
L_N_plus = Build_Cdag_Matrix(N, N)
L_N_minus = Build_C_Matrix(N, N) 

L_k_list = Matrix{ComplexF64}[L_1_plus, L_1_minus, L_N_plus, L_N_minus]
gamma_k_list = [Γ*(1+f)/8, Γ*(1-f)/8, Γ*(1-f)/8, Γ*(1+f)/8];

# L_k_list = Matrix{ComplexF64}[sqrt(Γ*(1+f)/8)*L_1_plus, sqrt(Γ*(1-f)/8)*L_1_minus, sqrt(Γ*(1-f)/8)*L_N_plus, sqrt(Γ*(1+f)/8)*L_N_minus]
# gamma_k_list = [1,1,1,1]

In [151]:
#Different Hamiltonian for each E.
E_values = LinRange(0, 12, 5)


@show E = E_values[1]
μ = -E*(N+1)/4

H = Build_H(N, Δ, μ, E);
Liouvillian_Superoperator = Build_Liouvillian_Superoperator(H, gamma_k_list, L_k_list)

E = E_values[1] = 0.0


256×256 Matrix{ComplexF64}:
 -0.25+0.0im     0.0+0.0im    0.0+0.0im  …     0.0+0.0im    0.0+0.0im
   0.0+0.0im  -0.375+2.5im    0.0-0.5im        0.0+0.0im    0.0+0.0im
   0.0+0.0im     0.0-0.5im  -0.25+5.0im        0.0+0.0im    0.0+0.0im
   0.0+0.0im     0.0+0.0im    0.0+0.0im        0.0+0.0im    0.0+0.0im
   0.0+0.0im     0.0+0.0im    0.0-0.5im        0.0+0.0im    0.0+0.0im
   0.0+0.0im     0.0+0.0im    0.0+0.0im  …     0.0+0.0im    0.0+0.0im
   0.0+0.0im     0.0+0.0im    0.0+0.0im        0.0+0.0im    0.0+0.0im
   0.0+0.0im     0.0+0.0im    0.0+0.0im        0.0+0.0im    0.0+0.0im
   0.0+0.0im     0.0+0.0im    0.0+0.0im        0.0+0.0im    0.0+0.0im
   0.0+0.0im     0.0+0.0im    0.0+0.0im        0.0+0.0im    0.0+0.0im
   0.0+0.0im     0.0+0.0im    0.0+0.0im  …     0.0+0.0im    0.0+0.0im
   0.0+0.0im     0.0+0.0im    0.0+0.0im        0.0+0.0im    0.0+0.0im
   0.0+0.0im     0.0+0.0im    0.0+0.0im        0.0+0.0im    0.0+0.0im
      ⋮                                  ⋱                    

In [158]:
GS = nullspace(Liouvillian_Superoperator)
rho_ss = reshape(GS, size(H)) #Steady State
rho_ss = rho_ss./tr(rho_ss)

J0 = 2*Γ*f/(Γ^2 +16) #Equation A(15 from article)
JP_i = [real(tr(JP_OP*rho_ss))/J0 for JP_OP = JP_operators]
JP = mean(JP_i);

0.003014356603559526